In [1]:
#r "nuget: Microsoft.ML"
#load "./Modules/MLWrapper.fs"
open Microsoft.ML
open Microsoft.ML.Data
open System.IO
open System.Net
open FunctionalMl

Installing package Microsoft.ML.failed!

Unhandled exception: C:\Users\rwvernag\Projects\github\uci-ml.net-examples\fsharp\Modules\MLWrapper.fs (2,16)-(2,18) typecheck error The namespace 'ML' is not defined.
C:\Users\rwvernag\Projects\github\uci-ml.net-examples\fsharp\Modules\MLWrapper.fs (3,16)-(3,18) typecheck error The namespace 'ML' is not defined.
C:\Users\rwvernag\Projects\github\uci-ml.net-examples\fsharp\Modules\MLWrapper.fs (7,19)-(7,28) typecheck error The value or constructor 'MLContext' is not defined. Maybe you want one of the following:
   AppContext
C:\Users\rwvernag\Projects\github\uci-ml.net-examples\fsharp\Modules\MLWrapper.fs (9,25)-(9,39) typecheck error The type 'EstimatorChain' is not defined.
C:\Users\rwvernag\Projects\github\uci-ml.net-examples\fsharp\Modules\MLWrapper.fs (10,9)-(10,32) typecheck error A unique overload for method 'Append' could not be determined based on type information prior to this program point. A type annotation may be needed. Candidates: (extension).Append<'TTag when 'TTag :> 

In [2]:
if not <| File.Exists("abalone.data") then
    use client = new WebClient()
    client.DownloadFile("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data", "abalone.data")

File.ReadLines("abalone.data")
|> Seq.take 5

Unhandled exception: input.fsx (1,11)-(1,15) typecheck error The value, namespace, type or module 'File' is not defined. Maybe you want one of the following:
   Failure
input.fsx (2,22)-(2,31) typecheck error The type 'WebClient' is not defined.
input.fsx (3,5)-(3,24) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (5,1)-(5,5) typecheck error The value, namespace, type or module 'File' is not defined. Maybe you want one of the following:
   Failure

In [22]:
[<CLIMutable>]
type AbaloneData =
    {
        [<LoadColumn(0)>]
        Sex : string

        [<LoadColumn(1)>]
        Length : float32

        [<LoadColumn(2)>]
        Diameter : float32

        [<LoadColumn(3)>]
        Height : float32

        [<LoadColumn(4)>]
        WholeWeight : float32

        [<LoadColumn(5)>]
        ShuckedWeight : float32

        [<LoadColumn(6)>]
        VisceraWeight : float32

        [<LoadColumn(7)>]
        ShellWeight : float32

        [<LoadColumn(8)>]
        [<ColumnName("Label")>]
        Rings : single
    }

let allData = ML.context.Data.LoadFromTextFile<AbaloneData>("abalone.data", hasHeader = false, separatorChar = ',')

In [24]:
let trainData, testData =
    ML.shuffle allData
    |> ML.split 0.2

In [25]:
let featureColumns = [| "Sex"; "Length"; "Diameter"; "Height"; "WholeWeight"; "ShuckedWeight"; "VisceraWeight"; "ShellWeight" |]

In [26]:
let pipeline = 
    EstimatorChain()
    |> ML.append <| ML.onehot "Sex" // one-hot encode the Sex feature
    |> ML.append <| ML.concatenate "Features" featureColumns // Concatenate feature columns into a single new column
    |> ML.append <| ML.normalizeLp "Features" "FeaturesNorm" // Normalize features into a new column, FeaturesNorm

In [27]:
let transformer =
    pipeline
    |> ML.fit trainData // Fit our pipeline on the training data

In [29]:
let transformedData =
    trainData
    |> ml.Transform transformer

Print the data as it was loaded from the file:

In [31]:
ML.context.Data.CreateEnumerable<AbaloneData>(trainData, reuseRowObject = false)
|> Seq.take 3

index,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,I,0.4,0.3,0.11,0.315,0.109,0.067,0.12,9
1,I,0.325,0.25,0.08,0.1735,0.0765,0.0345,0.049,7
2,F,0.46,0.355,0.13,0.517,0.2205,0.114,0.165,9


Compare that to the data as transformed by the pipeline. First, we one-hot encoded the `Sex` column. Then we concatenated all of the feature columns into a single new vector column, `Features`. Lastly, we normalized the values and put them into a new vector column, `FeaturesNorm`. Notice that the first three values of `Features` are the one-hot encoded values of `Sex`.

In [36]:
[<CLIMutable>]
type AbaloneDataTransformed =
    {
        [<ColumnName("Label")>]
        Rings : single

        [<VectorType(10)>]
        Features : single[]

        [<VectorType(10)>]
        FeaturesNorm : single[]
    }

ML.context.Data.CreateEnumerable<AbaloneDataTransformed>(transformedData, reuseRowObject = false)
|> Seq.take 3

index,Rings,Features,FeaturesNorm
0,9,"[ 1, 0, 0, 0.4, 0.3, 0.11, 0.315, 0.109, 0.067, 0.12 ]","[ 0.84755045, 0, 0, 0.3390202, 0.25426516, 0.093230546, 0.26697838, 0.092383, 0.05678588, 0.10170605 ]"
1,7,"[ 1, 0, 0, 0.325, 0.25, 0.08, 0.1735, 0.0765, 0.0345, 0.049 ]","[ 0.90756553, 0, 0, 0.2949588, 0.22689138, 0.072605245, 0.15746263, 0.069428764, 0.03131101, 0.04447071 ]"
2,9,"[ 0, 1, 0, 0.46, 0.355, 0.13, 0.517, 0.2205, 0.114, 0.165 ]","[ 0, 0.7645726, 0, 0.3517034, 0.27142328, 0.09939444, 0.39528406, 0.16858827, 0.08716128, 0.12615448 ]"


Create an estimator:

In [35]:
let estimator = 
    ML.context.Regression.Trainers.LbfgsPoissonRegression(featureColumnName = "FeaturesNorm")
    |> ml.DowncastEstimator

Now, we use cross-validation to select the best performing model.

In [38]:
let model =
    trainData // Begin with the training data
    |> ML.transform transformer // Transform using the transformer built above
    |> ML.crossValidateRegression estimator 3 // 3-fold cross-validation
    |> ML.printRegressionCvMetrics // Print cross-fold metrics
    |> Seq.maxBy (fun cvResult -> cvResult.Metrics.RSquared) // Select the best model by R-squared
    |> fun cvResult -> cvResult.Model

------------------
Cross Validation Metrics
------------------
Mean Absolute Error: 1.545188
Mean Squared Error: 4.630495
Root Mean Squared Error: 2.151663
R-squared: 0.558769


Now we can evaluate our model against the test data.

In [39]:
model
|> ML.transform <| ML.transform transformer testData // Transform the test data and get predictions
|> ML.context.Regression.Evaluate // Get test metrics
|> ML.printRegressionMetrics

------------------
Test Metrics
------------------
Mean Absolute Error: 1.500183
Mean Squared Error: 4.187195
Root Mean Squared Error: 2.046264
R-squared: 0.575770


Now let's pretend we have new data (for convenience we are just randomly re-sampling the test data) to see what predictions our model makes. You will see two values:
- `Label`: the actual number of rings from the example being predicted. Our model never sees this value but we show it below so that you can see how close the predicted number of rings are to the actual.
- `Score`: the predicted number of rings made by the model. The closer this is to the Label, the more accurate is the prediction.

You can run this cell multiple times to get new random samples and their predictions!

In [46]:
[<CLIMutable>]
type RegressionPrediction = { Label : single; Score : single }

// Show some sample predictions
let sampleData =
    testData
    |> ml.Shuffle 
    |> ml.Transform transformer

let predictionEngine = ML.context.Model.CreatePredictionEngine<AbaloneDataTransformed, RegressionPrediction>(model)

ML.context.Data.CreateEnumerable<AbaloneDataTransformed>(sampleData, reuseRowObject = false)
|> Seq.take 5
|> Seq.map predictionEngine.Predict

index,Label,Score
0,12,11.201082
1,10,9.523389
2,11,11.18794
3,6,6.242352
4,7,8.6737995
